Pre-trained model download link : [here](https://drive.google.com/file/d/1UDCJ3YllR3RN4QK8NTe5UoTP_s3nbde9/view?usp=sharing)

In [1]:
import os
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
!pip install tensorflow==1.12.0

     |████████████████████████████████| 83.1MB 55kB/s 
     |████████████████████████████████| 3.1MB 47.0MB/s 
     |████████████████████████████████| 51kB 8.7MB/s 
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [10]:
#!/usr/bin/env python

import argparse
import io
import os
import sys

import tensorflow as tf

#SCRIPT_PATH = os.path.dirname(os.path.abspath(__file__))

# Default paths.
'''DEFAULT_LABEL_FILE = os.path.join(
    SCRIPT_PATH, '/gdrive/My Drive/CS470/teamai/labels/2350-common-hangul.txt'
)
DEFAULT_GRAPH_FILE = os.path.join(
    SCRIPT_PATH, '/gdrive/My Drive/CS470/teamai/saved-model/optimized_hangul_tensorflow.pb'
)'''


def read_image(file):
    """Read an image file and convert it into a 1-D floating point array."""
    file_content = tf.read_file(file)
    image = tf.image.decode_jpeg(file_content, channels=1)
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    image = tf.reshape(image, (1, 64*64))
    return image


def classify(img_file, label_file, graph_file):
    """Classify a character.

    This method will import the saved model from the given graph file, and will
    pass in the given image pixels as input for the classification. The top
    five predictions will be printed.
    """
    labels = io.open(label_file,
                     'r', encoding='utf-8').read().splitlines()

    if not os.path.isfile(img_file):
        print('Error: Image %s not found.' % img_file)
        sys.exit(1)

    # Load graph and parse file.
    with tf.gfile.GFile(graph_file, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    with tf.Graph().as_default() as graph:
        tf.import_graph_def(
            graph_def,
            input_map=None,
            return_elements=None,
            name='hangul-model',
            producer_op_list=None
        )

    # Get relevant nodes.
    x = graph.get_tensor_by_name('hangul-model/input:0')
    y = graph.get_tensor_by_name('hangul-model/output:0')
    keep_prob = graph.get_tensor_by_name('hangul-model/keep_prob:0')

    image = read_image(img_file)
    sess = tf.InteractiveSession()
    image_array = sess.run(image)
    sess.close()
    with tf.Session(graph=graph) as graph_sess:
        predictions = graph_sess.run(y, feed_dict={x: image_array,
                                                   keep_prob: 1.0})
        prediction = predictions[0]

    # Get the indices that would sort the array, then only get the indices that
    # correspond to the top 5 predictions.
    sorted_indices = prediction.argsort()[::-1][:5]
    for index in sorted_indices:
        label = labels[index]
        confidence = prediction[index]
        return label

#change the path names in classify for your own file
# second arguement should be the label file, third arguement should be the pre-trained model
def tag(image_file):
    return classify(image_file, '/gdrive/My Drive/CS470/teamai/labels/2350-common-hangul.txt', '/gdrive/My Drive/CS470/teamai/saved-model/optimized_hangul_tensorflow.pb')  


In [4]:
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import glob
import pandas as pd

In [5]:
#the path in glob should be the path to the image files, also change png into the img files state
images = glob.glob('/gdrive/MyDrive/CS470/teamai/h_letter/h_weird/*.png')

In [18]:
#this will save the images with the tagged unicode into the directory you insert
for img in images:
  image = img
  cvimg = cv2.imread(image)
  dst = cv2.bitwise_not(cvimg)
  cv2.imwrite( "checking.jpg", dst )
  newimg = Image.open("checking.jpg")
  resized = newimg.resize((64,64))

  data = np.array(resized)
  red, green, blue = data[:,:,0], data[:,:,1],data[:,:,2]
  for i in range(64):
    for j in range(64):
      imsi = data[i][j]
      if (int(imsi[0])+int(imsi[1])+int(imsi[2])) < 300:
        data[i][j] = [0,0,0]
  newimg = Image.fromarray(data)

  newimg.save('new1.jpeg')
  letter = tag('/content/new1.jpeg')
  uni = 'uni' + str(hex(ord(letter)))[2:].upper()
  #the path will be the directory you want to save the img files
  path = '/gdrive/MyDrive/CS470/teamai/new_external/save_files/'
  cvimg = cv2.imread('new1.jpeg')
  dst = cv2.bitwise_not(cvimg)
  cv2.imwrite( path+uni+'.jpg', dst )
  

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
